In [3]:
from fastbook import *
from pathlib import Path
# Define the search queries for woman and man portraits
searches = ['woman pictures portrait', 'man pictures portrait']

# Set the path to the directory where the dataset will be stored
path = Path('dataset')

# Check if the dataset directory doesn't exist
if not path.exists():
    # Create the dataset directory if it doesn't exist
    path.mkdir(exist_ok=True)
    
    # Loop through the search queries
    for o in searches:
        # Create a subdirectory within the dataset directory for each search query
        x = o.split()[0]
        dest = (path/x)
        dest.mkdir(exist_ok=True)
        
        # Search and download images based on the current query
        results = search_images_ddg(o)
        download_images(dest, urls=results[:200])
        
        # Resize the downloaded images to a maximum size of 400 pixels
        resize_images(dest, max_size=400, dest=dest)

# Verify the integrity of the downloaded images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

# Define how the image data will be loaded and processed
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.4, seed=42),
    get_y=parent_label,
    item_tfms= Resize(128), batch_tfms=aug_transforms(mult=2)
).dataloaders(path)

# Create a CNN learner using the data loaders and the resnet18 architecture
learn = vision_learner(dls, resnet101, metrics=error_rate)


# Fine-tune the pre-trained model on the dataset for 10 epochs
learn.fine_tune(10)

# Save the trained model
learn.export('model.pkl')


C:\Users\urooj\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\urooj\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\urooj/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth
100%|███████████████████████████████████████████████████████████████████████████████| 171M/171M [00:36<00:00, 4.87MB/s]


epoch,train_loss,valid_loss,error_rate,time
0,1.070495,0.752594,0.215278,00:38


epoch,train_loss,valid_loss,error_rate,time
0,0.533960,0.537522,0.145833,00:47
1,0.503072,0.280052,0.118056,00:51
2,0.431693,0.288665,0.118056,00:51
3,0.413674,0.371385,0.097222,00:49
4,0.385581,0.416518,0.076389,00:49
5,0.354754,0.308456,0.076389,00:47
6,0.338012,0.265261,0.083333,00:50
7,0.304137,0.264374,0.076389,00:54
8,0.289190,0.265764,0.076389,00:53
9,0.279593,0.270595,0.069444,00:54
